In [ ]:
#milestone project one - food vision big
!nvidia-smi -L

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import make_confusion_matrix, pred_and_plot, plot_loss_curves, compare_historys, unzip_data, walk_through_dir, create_tensorboard_callback

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
import tensorflow_datasets as tfds
datasets_list = tfds.list_builders()
print('food101' in datasets_list)

In [ ]:
(train_data, test_data), ds_info = tfds.load(name = 'food101',
                                             split = ['train', 'validation'],
                                             shuffle_files = True,
                                             as_supervised = True,
                                             with_info = True)

In [ ]:
#features of food 101
class_names = ds_info.features['label'].names


In [ ]:
sample = train_data.take(1)
sample

In [ ]:
for image, label in sample:
  print(image.shape)
  print(image.dtype)
  print(label)
  print(class_names[label.numpy()])

In [ ]:
plt.figure()
plt.imshow(image)
plt.axis(False)

In [ ]:
def preprocess_img(image,label,img_shape = 224):
  """ converts image datatype from uint8 to float32 and reshapes image to imgshape"""
  image = tf.image.resize(image,[img_shape, img_shape])
  #you have to rescale if using a particular pretrained model
  return tf.cast(image,tf.float32), label

In [ ]:
preprocessed_image = preprocess_img(image,label)[0]
print(image[:2], image.dtype)
print(preprocessed_image[:2], preprocessed_image.dtype)


In [ ]:
#Map preprocesssing function to training (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls = tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)

test_data = test_data.map(map_func=preprocess_img, num_parallel_calls = tf.data.AUTOTUNE).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)


In [ ]:
train_data,test_data

In [ ]:
#create tensorboard callback

from helper_functions import create_tensorboard_callback

checkpoint_path = 'model_checkpoint/cp.ckpt'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor = 'vall_acc',
                                                      save_best_only = True,
                                                      save_weights_only = True,
                                                      verbose = 0)


In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

inputs = layers.Input(shape = input_shape, name = 'input_layer')
#dont need to rescale - efficientnet
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation('softmax', dtype=tf.float32, name = 'output_layer')(x)
model = tf.keras.Model(inputs,outputs)
model.compile(loss = 'SparseCategoricalCrossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Turn off all warnings except for errors
tf.get_logger().setLevel('ERROR')

# Fit the model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data, 
                                                     epochs=3,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs", 
                                                                                            "efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

Saving TensorBoard log files to: training_logs/efficientnetb0_101_classes_all_data_feature_extract/20221221-100013
Epoch 1/3
  24/2368 [..............................] - ETA: 102:04:46 - loss: nan - accuracy: 0.0026